# **Setup**

In [1]:
import gradio as gr
import torch
import carClassifier
import LLM_JSON
import json
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import langchain 

# **Image Classification**

In [2]:
#Load the image classification model
device = torch.device('cpu')
car_model = carClassifier.CarTypeClassifier(10)
car_model = torch.jit.load('car_model_scripted.pt', map_location=device)

In [3]:
#Get class names and indices
f = open('Classes Dict.json')
classes = json.load(f)

f.close()
print(classes)

{'0': 'Cab', '1': 'Convertible', '2': 'Coupe', '3': 'Hatchback', '4': 'Minivan', '5': 'SUV', '6': 'Sedan', '7': 'SuperCab', '8': 'Van', '9': 'Wagon'}


In [4]:
# #Apply transforms to image before prediction
def ApplyTransform(image):
    #batch_size=1
    IMG_SIZE = (480, 480)
    size = 480
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    predict_transforms = transforms.Compose([
        transforms.CenterCrop(size=size),
         transforms.Resize(IMG_SIZE),
         transforms.Grayscale(num_output_channels=3),
         transforms.ToTensor(),
         transforms.Normalize(mean, std)
    ])
    
    # test_dataloader = DataLoader(
    #     predict_transforms(image),
    #     batch_size=batch_size
    # )
    
    return predict_transforms(image)

In [5]:
# def predict(image):
#     print(image.shape)
#     image_tensor=ApplyTransform(image)
#     print(image_tensor.size())

#     car_model.eval()
#     with torch.no_grad():
#         print(image_tensor.size())
#         output = car_model(image_tensor)
#         _, pred = torch.max(output, 1)
#         print(pred)
#         body_type=classes[pred]
#         return body_type

def predict(image):
  return 'car body type'    

# **LLM**

In [6]:
llm_model = LLM_JSON.LLM()

D:\Ash'Ooosha\Software\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
D:\Ash'Ooosha\Software\Python\Python310\lib\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://orionopenai.openai.azure.com/ to https://orionopenai.openai.azure.com/openai.
  warnings.warn(
D:\Ash'Ooosha\Software\Python\Python310\lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_

# **Gradio**

In [7]:
def car_description(text, image):
    
    #predict body type
    body_type = predict(image)
        
    #extract car attributes
    llm_json = llm_model.llm_response(text)
    
    #Add body type to extracted attributes
    new_dictionary = {"body_type": body_type}
    new_dictionary.update(llm_json['car'])
    new_dict = {'car':new_dictionary}
    
    return new_dict

In [8]:
langchain.debug = True 
interface = gr.Interface(fn=car_description, inputs=[gr.Textbox(lines=5, placeholder="Type car description"), gr.components.Image()],
                     outputs=gr.JSON(), title="Online car-selling platform",
                     description="Display car attributes in JSON format")

# Launch the interface
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://1195dde175d05679ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
